<a href="https://colab.research.google.com/github/ahnjonghyunn/ABC/blob/main/%EC%A7%80%EC%A7%84_%EC%96%BC%EB%A7%88%EB%82%98_%EC%9E%90%EC%A3%BC_%EB%B0%9C%EC%83%9D%ED%95%98%EB%82%98_%EC%95%8C%EC%95%84%EB%B3%B4%EC%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

필요한 패키지 다운로드 밎 임포트

In [ ]:
!pip install koreanize-matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import koreanize_matplotlib
import folium

데이터 불러오기(전국 지진데이터)

In [ ]:
# 2006/01/01~2023/07/17 기간동안 발생한 지진데이터 불러오기
origin=pd.read_csv('/content/EQK_2000-01-01_2022-12-31.csv',encoding='cp949')
origin.head()

,번호,발생시각,규모,깊이(km),최대,위도,경도,위치,지도보기,상세정보
0,NaN,NaN,NaN,NaN,진도,NaN,NaN,NaN,NaN,(진도 등)
1,1662.0,2022-12-25 19:31,2.6,14,Ⅲ,36.51 N,128.65 E,경북 안동시 남서쪽 10km 지역,지도,상세정보
2,1661.0,2022-12-23 19:43,2.3,-,Ⅰ,35.14 N,125.08 E,전남 신안군 흑산도 북북서쪽 61km 해역,지도,상세정보
3,1660.0,2022-12-16 12:07,2.1,26,Ⅰ,41.30 N,129.22 E,북한 함경북도 길주 북북서쪽 39km 지역,지도,상세정보
4,1659.0,2022-12-03 13:22,2.9,21,Ⅰ,35.77 N,129.80 E,울산 북구 동북동쪽 45km 해역,지도,상세정보


In [ ]:
origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1663 entries, 0 to 1662
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   번호      1662 non-null   float64
 1   발생시각    1662 non-null   object 
 2   규모      1662 non-null   float64
 3   깊이(km)  985 non-null    object 
 4   최대      349 non-null    object 
 5   위도      1662 non-null   object 
 6   경도      1662 non-null   object 
 7   위치      1662 non-null   object 
 8   지도보기    1662 non-null   object 
 9   상세정보    1663 non-null   object 
dtypes: float64(2), object(8)
memory usage: 130.0+ KB


데이터 전처리(필요 정보만 뽑아내는 과정)

In [ ]:
df=origin.drop(0)
df.head()

,번호,발생시각,규모,깊이(km),최대,위도,경도,위치,지도보기,상세정보
1,1662.0,2022-12-25 19:31,2.6,14,Ⅲ,36.51 N,128.65 E,경북 안동시 남서쪽 10km 지역,지도,상세정보
2,1661.0,2022-12-23 19:43,2.3,-,Ⅰ,35.14 N,125.08 E,전남 신안군 흑산도 북북서쪽 61km 해역,지도,상세정보
3,1660.0,2022-12-16 12:07,2.1,26,Ⅰ,41.30 N,129.22 E,북한 함경북도 길주 북북서쪽 39km 지역,지도,상세정보
4,1659.0,2022-12-03 13:22,2.9,21,Ⅰ,35.77 N,129.80 E,울산 북구 동북동쪽 45km 해역,지도,상세정보
5,1658.0,2022-12-01 15:17,3.2,14,Ⅲ,36.17 N,128.27 E,경북 김천시 동북동쪽 14km 지역,지도,상세정보


In [ ]:
#지진 데이터 분석을 위한 컬럼만 추출
eq=df.drop(['번호','지도보기','최대','깊이(km)','상세정보'],axis=1)
eq.head()

,발생시각,규모,위도,경도,위치
1,2022-12-25 19:31,2.6,36.51 N,128.65 E,경북 안동시 남서쪽 10km 지역
2,2022-12-23 19:43,2.3,35.14 N,125.08 E,전남 신안군 흑산도 북북서쪽 61km 해역
3,2022-12-16 12:07,2.1,41.30 N,129.22 E,북한 함경북도 길주 북북서쪽 39km 지역
4,2022-12-03 13:22,2.9,35.77 N,129.80 E,울산 북구 동북동쪽 45km 해역
5,2022-12-01 15:17,3.2,36.17 N,128.27 E,경북 김천시 동북동쪽 14km 지역


In [ ]:
eq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1662 entries, 1 to 1662
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   발생시각    1662 non-null   object 
 1   규모      1662 non-null   float64
 2   위도      1662 non-null   object 
 3   경도      1662 non-null   object 
 4   위치      1662 non-null   object 
dtypes: float64(1), object(4)
memory usage: 65.1+ KB


지역별 지진 데이터 모아보기

In [ ]:
location_check = eq['위치'].str.split(' ').str[0]
location_check

1       경북
2       전남
3       북한
4       울산
5       경북
        ..
1658    충남
1659    충남
1660    전북
1661    경북
1662    북한
Name: 위치, Length: 1662, dtype: object

In [ ]:
#첫번째 지역명 추출->17개 지역으로 나눠서 지진 확인
location_list = np.sort(location_check.unique()) # 가,나,다 순으로정렬
location_list

array(['강원', '경기', '경남', '경북', '광주', '대구', '대전', '부산', '북한', '서울', '울산',
       '인천', '전남', '전북', '제주', '충남', '충북'], dtype=object)

In [ ]:
# 각 지역의 정보를 변수명으로 만들고 지진데이터 모아보기
for loc in location_list:
 globals()[str(loc)] = eq[eq['위치'].str.contains(loc)]


In [ ]:
# 경북 지진 데이터에서 북한을 제외하기
경북 = 경북[~경북['위치'].str.contains('북한')]
경북

,발생시각,규모,위도,경도,위치
1,2022-12-25 19:31,2.6,36.51 N,128.65 E,경북 안동시 남서쪽 10km 지역
5,2022-12-01 15:17,3.2,36.17 N,128.27 E,경북 김천시 동북동쪽 14km 지역
24,2022-09-11 16:28,2.4,36.46 N,129.63 E,경북 영덕군 동북동쪽 24km 해역
26,2022-09-03 0:44,2.0,36.39 N,129.89 E,경북 영덕군 동쪽 47km 해역
29,2022-08-24 18:31,2.1,36.47 N,129.62 E,경북 영덕군 동북동쪽 24km 해역
...,...,...,...,...,...
1635,2000-12-09 18:51,3.7,36.50 N,130.00 E,경북 영덕군 동쪽 57km 해역
1647,2000-08-06 19:35,2.2,36.30 N,128.40 E,경북 의성군 서남서쪽 27km 지역
1651,2000-05-19 1:44,2.7,36.60 N,128.50 E,경북 안동시 서쪽 21km 지역
1657,2000-04-15 8:05,2.3,36.60 N,128.30 E,경북 예천군 서남서쪽 15km 지역


지역벌 지진 규모와의 상관관계를 그래프로 시각화 하기

In [ ]:
# 지역별 지진 규모 상관관계 그래프 시각화
fig = px.scatter(eq, x='발생시각', y='규모',color='위치')
fig.show()

지진이 가장많이 발생한 지역(빈도수)시각화 하기

In [ ]:
from ast import Global
whole = pd.DataFrame({'지진발생수': [len(globals()[loc])for loc in location_list]},
                    index = location_list)
whole

,지진발생수
강원,86
경기,17
경남,90
경북,585
광주,5
대구,8
대전,8
부산,9
북한,261
서울,1


In [ ]:
fig = px.bar(whole,  y='지진발생수',text_auto=True)
fig.show()

대전,충북,충남, 데이터 모아서 시각화하기

In [ ]:
province = pd.concat((대전,충북,충남),sort=False)
province

,발생시각,규모,위도,경도,위치
320,2018-11-19 3:34,2.1,36.42 N,127.30 E,대전 유성구 북서쪽 8km 지역
414,2018-01-10 18:42,2.0,36.26 N,127.34 E,대전 서구 남남서쪽 12km 지역
667,2016-11-27 6:53,2.5,36.34 N,127.33 E,대전 유성구 남서쪽 3km 지역
1416,2006-03-19 14:03,2.0,36.33 N,127.34 E,대전 서구 서남서쪽 3km 지역
1417,2006-03-19 13:59,2.9,36.34 N,127.36 E,대전 서구 서남서쪽 1km 지역
...,...,...,...,...,...
1638,2000-11-05 21:39,2.2,36.20 N,127.20 E,충남 논산시 동북동쪽 9km 지역
1642,2000-10-03 0:01,2.0,36.70 N,127.20 E,충남 연기군 북서쪽 15km 지역
1650,2000-05-26 21:41,2.4,36.20 N,127.10 E,충남 논산시 북쪽 2km 지역
1658,2000-04-12 4:44,3.5,36.90 N,125.30 E,충남 태안군 서격렬비열도 북북서쪽 39km 해역


In [ ]:
fig = px.bar(province, x='위치' ,y='규모')
fig.show()

대전,충북,충남에서 지진규모 TOP20모아서 시각화

In [ ]:
top_20 = province.sort_values(by='규모',ascending=False).head(20)
top_20

,발생시각,규모,위도,경도,위치
977,2014-04-01 4:48,5.1,36.95 N,124.50 E,충남 태안군 서격렬비도 서북서쪽 100km 해역
17,2022-10-29 8:27,4.1,36.88 N,127.88 E,충북 괴산군 북동쪽 11km 지역
1521,2003-06-09 10:14,4.0,36.00 N,123.60 E,충남 태안군 서격렬비열도 서남서쪽 186km 해역
1508,2003-10-13 18:12,3.6,37.00 N,126.50 E,충남 당진군 북서쪽 17km 지역
1027,2013-07-13 2:02,3.5,36.20 N,126.18 E,충남 보령시 서남서쪽 42km 해역
18,2022-10-29 8:27,3.5,36.88 N,127.88 E,충북 괴산군 북동쪽 11km 지역
674,2016-11-13 21:52,3.5,36.36 N,126.63 E,충남 보령시 북북동쪽 4km 지역
1658,2000-04-12 4:44,3.5,36.90 N,125.30 E,충남 태안군 서격렬비열도 북북서쪽 39km 해역
1301,2008-10-29 9:26,3.4,36.35 N,127.25 E,충남 공주시 남동쪽 15km 지역
1275,2009-03-02 14:20,3.4,37.08 N,124.65 E,충남 태안군 서격렬비도 북서쪽 95km 해역


In [ ]:
fig = px.bar(top_20, x='위치' ,y='규모',text_auto=True)
fig.show()

전국 지진 데이터를 지도에 시각화 하기

In [ ]:
eq_df = eq.sort_values(by='규모')
eq_df

,발생시각,규모,위도,경도,위치
1496,2004-04-15 23:25,2.0,34.90 N,124.60 E,전남 신안군 흑산면 서북서쪽 84km 해역
450,2017-11-22 22:15,2.0,36.12 N,129.35 E,경북 포항시 북구 북쪽 9km 지역
759,2016-09-13 12:33,2.0,35.76 N,129.16 E,경북 경주시 남남서쪽 10km 지역
445,2017-11-25 12:51,2.0,36.10 N,129.35 E,경북 포항시 북구 북쪽 7km 지역
761,2016-09-13 10:58,2.0,35.76 N,129.18 E,경북 경주시 남남서쪽 9km 지역
...,...,...,...,...,...
844,2016-09-12 19:44,5.1,35.77 N,129.19 E,경북 경주시 남남서쪽 8.2km 지역
977,2014-04-01 4:48,5.1,36.95 N,124.50 E,충남 태안군 서격렬비도 서북서쪽 100km 해역
1486,2004-05-29 19:14,5.2,36.80 N,130.20 E,경북 울진군 동남동쪽 74km 해역
513,2017-11-15 14:29,5.4,36.11 N,129.37 E,경북 포항시 북구 북쪽 8km 지역


지도 시각화를 위한 위도,경도 전처리

In [ ]:
eq_df[['위도','경도']]

,위도,경도
1496,34.90 N,124.60 E
450,36.12 N,129.35 E
759,35.76 N,129.16 E
445,36.10 N,129.35 E
761,35.76 N,129.18 E
...,...,...
844,35.77 N,129.19 E
977,36.95 N,124.50 E
1486,36.80 N,130.20 E
513,36.11 N,129.37 E


In [ ]:
eq_df['위도']=eq_df['위도'].str.split(' ').str[0].astype('float')
eq_df['경도']=eq_df['경도'].str.split(' ').str[0].astype('float')
eq_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1662 entries, 1496 to 829
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   발생시각    1662 non-null   object 
 1   규모      1662 non-null   float64
 2   위도      1662 non-null   float64
 3   경도      1662 non-null   float64
 4   위치      1662 non-null   object 
dtypes: float64(3), object(2)
memory usage: 77.9+ KB


In [ ]:
eq_df[['위도','경도']]

,위도,경도
1496,34.90,124.60
450,36.12,129.35
759,35.76,129.16
445,36.10,129.35
761,35.76,129.18
...,...,...
844,35.77,129.19
977,36.95,124.50
1486,36.80,130.20
513,36.11,129.37


지진 지도 시각화

In [ ]:
center = [37.541,126.986]# 서울 시청 좌표
map = folium.Map(location=center,zoom_start=6)
# 지진 위치에 cicle 표시-> 지진 규모애 따라 cicle 크기 지정
for id,geo_df_row in eq_df.iterrows():
  folium.Circle(location=[geo_df_row['위도'],geo_df_row['경도']],
                popup=str(geo_df_row['위치']),
                color='green',
                fill_color='green',
                weigh=0.5,
                radius=float(geo_df_row['규모']**7)
                ).add_to(map)
map

제주도에서 발생한 지진데이터 분석

In [ ]:
# 광주 지진 데이터 가져오기
eq_gwangju = eq_df[eq_df['위치'].str.contains('광주')]
eq_gwangju

,발생시각,규모,위도,경도,위치
598,2017-03-27 9:28,2.0,35.21,126.95,광주 북구 북동쪽 5km 지역
696,2016-10-09 14:03,2.2,35.08,126.93,광주 동구 남쪽 7km 지역
961,2014-08-02 1:32,2.2,37.40,127.20,경기 광주시 서남서쪽 5km 지역
619,2017-02-18 20:58,2.4,35.23,126.94,광주 북구 북북동쪽 7km 지역
1046,2013-06-08 5:56,3.2,35.14,126.96,광주 동구 동쪽 3km 지역


In [ ]:
center = [35.1595454,126.8526012]# 광주시청 좌표
map = folium.Map(location=center,zoom_start=6)
# 지진 위치에 cicle 표시-> 지진 규모애 따라 cicle 크기 지정
for id,geo_df_row in eq_gwangju.iterrows():
  folium.Circle(location=[geo_df_row['위도'],geo_df_row['경도']],
                popup=str(geo_df_row['위치']),
                color='gray',
                fill_color='gray',
                weigh=0.5,
                radius=float(geo_df_row['규모']**7)
                ).add_to(map)

# 마커 찍기
folium.Marker(location=center,
              popup='광주시청',
              icon=folium.Icon(icon='home',prefix='fa',color='orange')).add_to(map)
map